In [117]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time
from IPython.display import Image

In [118]:
# Use my file:
path_to_file = "himym.txt"

In [119]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding = "ISO-8859-1")
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 15391 characters


In [120]:
# The unique characters in the file
vocab = sorted(set(text))

In [121]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [122]:
# The maximum length sentence we want for a single input in characters
seq_length = 120
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [123]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [124]:
# Batch size
BATCH_SIZE = 100

# How many individual training steps per samples (steps)
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [125]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [126]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
  rnn2 = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [127]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
      
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),

    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [128]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

W1019 13:20:50.111489 139965152700224 util.py:244] Unresolved object in checkpoint: (root).optimizer
W1019 13:20:50.112749 139965152700224 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer
W1019 13:20:50.113587 139965152700224 util.py:244] Unresolved object in checkpoint: (root).optimizer.global_step
W1019 13:20:50.114210 139965152700224 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer.beta1_power
W1019 13:20:50.114840 139965152700224 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer.beta2_power
W1019 13:20:50.115468 139965152700224 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer's state 'm' for (root).layer_with_weights-0.embeddings
W1019 13:20:50.116108 139965152700224 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer's state 'm' for (root).layer_with_weights-1.kernel
W1019 13:20:50.116725 139965152700224 util.py:244] Unresolved object in checkpoint: (root).optimizer.

In [129]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)

In [130]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [131]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (100, 120, 69)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.233464


In [132]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [133]:
# Directory where the checkpoints will be saved
checkpoint_dir = './CheckpointData'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [134]:
history = model.fit(dataset.repeat(), epochs=100, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 4.2335
Epoch 2/100
1/1 [==============================] - 1s 550ms/step - loss: 4.1530
Epoch 3/100
1/1 [==============================] - 1s 569ms/step - loss: 3.7195
Epoch 4/100
1/1 [==============================] - 1s 547ms/step - loss: 11.0283
Epoch 5/100
1/1 [==============================] - 1s 598ms/step - loss: 3.6927
Epoch 6/100
1/1 [==============================] - 1s 555ms/step - loss: 3.7492
Epoch 7/100
1/1 [==============================] - 1s 572ms/step - loss: 3.9543
Epoch 8/100
1/1 [==============================] - 1s 584ms/step - loss: 3.9997
Epoch 9/100
1/1 [==============================] - 1s 582ms/step - loss: 3.9941
Epoch 10/100
1/1 [==============================] - 1s 606ms/step - loss: 3.9624
Epoch 11/100
1/1 [==============================] - 1s 588ms/step - loss: 3.9081
Epoch 12/100
1/1 [==============================] - 1s 562ms/step - loss: 3.8361
Epoch 13/100
1/1 [=====================

In [135]:
tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [136]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.7

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
        
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [137]:
result = generate_text(model, start_string=u"Begin:\n")


In [138]:
# Save the result in a file
f = open('bachSize=100_epochs=100.txt', 'w+')
f.write(result)
f.close()